In [5]:
import numpy as np
import pandas as pd
import openpyxl

########################################part 1#####################################

# Lese die Excel-Datei mit dem Profil, welches varriert werden soll ein
# df_profil = pd.read_excel("profil_spring.xlsx")
# df_profil = pd.read_excel("profil_summer.xlsx")
# df_profil = pd.read_excel("profil_fall.xlsx")
df_profil = pd.read_excel("profil_winter.xlsx")

# Anzahl der Zufallsvariablen pro Zeitpunkt
num_variables = 32

# Erstelle ein leeres Pandas-DataFrame für die Ergebnisse
results = pd.DataFrame()

# Für jedes Szenario:   
for i in range(100):
    # Erstelle ein Array mit normalverteilten Zufallsvariablen für jeden Bus und jeden Zeitpunkt
    random_variables = np.random.normal(0, 0.005, (len(df_profil), num_variables))
    
    # Berechne die Summe der Zufallsvariablen für jeden Zeitpunkt
    sum_random_variables = np.sum(random_variables, axis=1)
       
    # Berechne den Faktor, um den jede Zufallsvariable angepasst werden muss, damit die Summe 0 ergibt
    factor = -sum_random_variables / num_variables

    # Passe jede Zufallsvariable an, indem der Faktor multipliziert wird
    random_variables_adjusted = random_variables + factor.reshape(-1, 1)

    # Die gewünschte Toleranz weil die Berechnung von Gleitkommazahlen ungenau ist und somit die Summe der angepassten Zufallszahlen minimal von 0 abweichen kann
    tolerance = 1e-10

    # Prüfe, ob die Summe der angepassten Zufallsvariablen für jeden Zeitpunkt innerhalb der Toleranz ist
    assert np.allclose(np.sum(random_variables_adjusted, axis=1), 0, atol=tolerance)
    
    # Füge die Zufallsvariablen ein
    df_varied = pd.DataFrame(random_variables_adjusted)

    # Setze die Spaltenüberschriften auf "Scenario i"
    df_varied.columns = ["S{}".format(i+1) for i in range(num_variables)]
    
    # Füge die veränderten Lastparameter dem DataFrame der Ergebnisse hinzu
    results = pd.concat([results, df_varied], axis=1)
      
# Speichere das DataFrame der Ergebnisse in einer Excel-Datei
results.to_excel("Ergebnisse.xlsx", index=False)

########################################part 2#####################################
#load_bus = pd.read_excel('load_bus_spring.xlsx')
#load_bus = pd.read_excel('load_bus_summer.xlsx')
#load_bus = pd.read_excel('load_bus_fall.xlsx')
load_bus = pd.read_excel('load_bus_winter.xlsx')

load_profil = pd.DataFrame(columns=['b'] + ['t'+str(i) for i in range(1,97)])
load_profil['b'] = load_bus['b']
for i in range(1,97):
    load_profil['t'+str(i)] = df_profil['profil'][i-1] * load_bus['Active demand']
    
########################################part 3#####################################

# Berechne die Lastabweichungen zum normalen Profil df_profil
load_variation = pd.DataFrame()

# Iteration über alle Szenarien
for scenario in results.columns:
    results[scenario] = results[scenario].fillna(0)
    load_variation_scenario = pd.DataFrame(columns=['Scenario','b'] + ['t'+ str(i) for i in range(1,97)])
    load_variation_scenario['Scenario'] = [scenario]*33
    load_variation_scenario['b'] = load_bus['b']
    for i in range(1,97):
        load_variation_scenario['t'+ str(i)] = results[scenario][i-1] * load_bus['Active demand'] - load_profil['t'+str(i)]
#        load_variation_scenario.subtract(load_profil)
    load_variation = pd.concat([load_variation,load_variation_scenario], ignore_index=True )
    
# umbennenung für Kompatibilität mit Gams   
load_variation.rename(columns={'t'+str(i): str(i) for i in range(1,97)}, inplace=True)
    
#load_variation.head()
load_variation.to_excel("Data.xlsx", index=False)

KeyError: 0